In [1]:
from helper import *

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = list({url['url'] for url in get_top_urls_for_google_search_query("ai conferences")})
markdowns = scrape_and_convert_to_markdown(urls[:10])
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
]
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
chunk_size = 500
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)
splits = []
for markdown_document in markdowns:
    md_header_splits = markdown_splitter.split_text(markdown_document)
    splits.extend(text_splitter.split_documents(md_header_splits))


In [ ]:
llm = llama_cpp.Llama(
  model_path="./models/mxbai-embed-large-v1-f16.gguf", 
  embedding=True, 
  verbose=False
)